intial caption


In [ ]:
!unzip 'drive/MyDrive/flickr.zip'

In [ ]:
def readTextfile(path):
  with open(path) as file:
    captions=file.read()
  return captions

In [ ]:
captions=readTextfile('captions.txt')

In [ ]:
caption_list=captions.split('\n')
caption_list=caption_list[:-1]
caption_list=caption_list[1:]

In [ ]:
descriptions={}

for x in caption_list:
  first,second= x.split('.jpg,')
  img_name=first

  if descriptions.get(img_name) is None:
    descriptions[img_name]=[]
  
  descriptions[img_name].append(second)
      

In [ ]:
import re

def clean_text(sentence):                           #not doing stemming lemmanization and stopword removal for human like output...
  sentence=sentence.lower()
  sentence=re.sub("[^a-z]"," ",sentence)
  sentence=sentence.split()

  sentence=" ".join(sentence)

  return sentence




In [ ]:
for key,cap in descriptions.items():
  for i in range(len(cap)):
    cap[i]=clean_text(cap[i])

In [ ]:
with open("drive/MyDrive/descriptions.txt",'w') as f:                         #saving in descriptions.txt
  f.write(str(descriptions))


**Vocab
**

In [ ]:
import json

descriptions=None

with open('/content/drive/MyDrive/descriptions.txt','r') as f:
  descriptions=f.read()

json_acceptable_string=descriptions.replace("'","\"")
descriptions=json.loads(json_acceptable_string)


In [ ]:
vocab=set()                                                      #vocab created

for cap in descriptions.values():
  for sentence in cap:
    vocab.update(sentence.split())





In [ ]:
import collections

total_words=[]
for key in descriptions.keys():
  for s in descriptions[key]:
    for word in s.split():
      total_words.append(word)


counter=collections.Counter(total_words)
freq_count=dict(counter)


In [ ]:
sorted_freq=sorted(freq_count.items(),reverse=True,key=lambda x:x[1]) 

In [ ]:
threshold=10                # min frequency for word's consideration

freq=[x for x in sorted_freq if x[1]>threshold ]

freq_vocab=[x[0] for x in freq]



In [ ]:
import pickle

f=open('/content/drive/MyDrive/encoded_train.pkl','rb')
train=pickle.load(f)
f.close()

train_descriptions={}
test=[]
                                             #for reloading the train/test set....not to be done multiple times

for key in descriptions.keys():

  if train.get(key) is not None:
    train_descriptions[key]=[]
    for sentence in descriptions[key]:
       train_descriptions[key].append("<s> "+sentence+" <e>")
  
  else:
    test.append(key)
  









###  Train/test
  




In [ ]:

                 #run ONLY ONCE

train_descriptions={}

test=[]
                                                                  # random train/test split (85/15)
import random

for key in descriptions.keys():
  choice=random.random()
  if choice <=0.80:
    train_descriptions[key]=[]

    for sentence in descriptions[key]:
      train_descriptions[key].append("<s> "+sentence+" <e>")
  
  else:
    test.append(key)





In [ ]:
print(len(train_descriptions))

In [ ]:
test=test[:1000] 
                                  # 1000 testing images

*image preprocessing *





In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model


In [ ]:
model=ResNet50(weights='imagenet',input_shape=(224,224,3))


In [ ]:
model.summary()

In [ ]:
model_new=Model(model.input,model.layers[-2].output)

In [ ]:
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
                                                                                     
def img_preprocess(img_path):
  img=image.load_img(img_path,target_size=(224,224,3))
  img=image.img_to_array(img)                                 
  img=img.reshape(-1,224,224,3)
# resnet 50 preprocessing function
  img=preprocess_input(img)
  return img



In [ ]:
def encode_img(img):
  img=img_preprocess(img)
  features=model_new.predict(img)

  feature_vec=features.reshape((-1,))
  return feature_vec


In [ ]:
train_encoded={}
IMG_PATH="Images"
c=0
for key in train_descriptions.keys():
  img_data=encode_img(IMG_PATH+"/"+key+".jpg")

  train_encoded[key]=img_data
  if c<100:
    c+=1
  else:
    print("one batch done")
    c=0

import pickle

with open("drive/MyDrive/encoded_train.pkl",'wb') as f:
  pickle.dump(train_encoded,f)  
  

In [ ]:
print(len(train_encoded))

In [ ]:
test_encoded={}
IMG_PATH="Images"
c=0
for t in test:
  img_data=encode_img(IMG_PATH+"/"+t+".jpg")
  test_encoded[t]=img_data

  if c%100==0:
    print(c/100)
  c+=1

with open("drive/MyDrive/encoded_test.pkl",'wb') as f:
  pickle.dump(test_encoded,f)


In [ ]:
import pickle
with open('/content/drive/MyDrive/encoded_train.pkl','rb') as f:
  encoded_train=pickle.load(f)




In [ ]:
train_descriptions={}
for t in encoded_train.keys():
  train_descriptions[t]=[]
  for s in descriptions[t]:
    train_descriptions[t].append("<s> "+s+" <e>")



# text preprocessing


In [ ]:
print(len(freq_vocab))

In [ ]:
vocab=[]
vocab=freq_vocab

word_idx={}
idx_word={}

for ix,word in enumerate(vocab):
  word_idx[vocab[ix]]=ix+1
  idx_word[ix+1]=vocab[ix]
  


In [ ]:
print(word_idx['cannon'])

In [ ]:
idx_word[1851]="<s>"
word_idx["<s>"]=1851
                                   #adding start seq and end seq to vocabulary
idx_word[1852]="<e>"
word_idx["<e>"]=1852

vocab_size=len(word_idx)+1

In [ ]:
import pickle

f=open('drive/MyDrive/word_idx.pkl','wb')
pickle.dump(word_idx,f)
f.close()

f=open('drive/MyDrive/idx_word.pkl','wb')
pickle.dump(idx_word,f)
f.close()



# data generator


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np


def data_generator(train_descriptions,encoded_train,word_idx,maxlen,batch_size):
  x1,x2,y= [],[],[]

  n=0
  while True:
    
    for key,data in train_descriptions.items():
      n+=1

      photo=encoded_train[key]
      for desc in data:
        seq=[word_idx[x] for x in desc.split() if x in word_idx]
        for i in range(1,len(seq)):
          xi=seq[0:i]                   #generating and padding
          yi=seq[i]
          
          xi=pad_sequences([xi],maxlen=maxlen,value=0,padding="post",truncating='post')[0]

          yi=to_categorical([yi],num_classes=len(word_idx)+1)[0]
      
          x1.append(photo)
          x2.append(xi)
          y.append(yi)
          

        if n==batch_size:
        
          yield([np.array(x1),np.array(x2)],np.array(y))

          n=0
          x1,x2,y= [],[],[]

        

          
      

# Glove embedding


In [ ]:
  import numpy as np


  f=open('/content/drive/MyDrive/glove.6B.50d.txt','r',encoding='utf-8')

  word_embedding={}

  lines=f.readlines()
  f.close()

  for line in lines:
    line=line.split()
    word=line[0]
    word_embedding[word]=np.array(line[1:],dtype='float')
  
  
    



In [ ]:
def form_embeddding_matrix(word_idx):
  embedd_dim=50
  vocab_size=len(word_idx)+1                         # 0 index refers to padding
  matrix=np.zeros((vocab_size,embedd_dim))

  for w,i in word_idx.items():
    embedding_values=word_embedding.get(w)
    if embedding_values is not None:
      matrix[i]=embedding_values

  return matrix

embedding_matrix=form_embeddding_matrix(word_idx)

Image caption model

In [ ]:
import tensorflow.keras

max_len=16

from tensorflow.keras.layers import *
from tensorflow.keras import Input

# img features input
input_img_features=Input(shape=(2048,))
inp_img1=Dropout(0.3)(input_img_features)
inp_img2=Dense(256,activation='relu')(inp_img1)

vocab_size=len(word_idx)+1
# caption input
input_captions=Input(shape=(max_len,))
inp_cap1=Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2=Dropout(0.3)(inp_cap1)
inp_cap3=LSTM(256)(inp_cap2)




In [ ]:
print(inp_img1.shape)
print(input_img_features.shape)

In [ ]:
#decoder

decoder1=add([inp_img2,inp_cap3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(vocab_size,activation='softmax')(decoder2)


In [ ]:
from tensorflow.keras.models import Model

model_x=Model(inputs=[input_img_features,input_captions],outputs=outputs)
model_x.summary()

In [ ]:
#setting embedding weights

model_x.layers[2].set_weights([embedding_matrix])
model_x.layers[2].trainable=False

In [ ]:
model_x.compile(optimizer='adam',loss='categorical_crossentropy')

Train>>>...


In [ ]:
model_x.load_weights('/content/drive/MyDrive/latest.h5')

In [ ]:


def train_model(epochs,batch_size):
  
  steps=len(encoded_train)//batch_size
  for i in range(epochs):
    generator=data_generator(train_descriptions,encoded_train,word_idx,16,3)
    model_x.fit_generator(generator,epochs=1,verbose=1,steps_per_epoch=steps)
    print('epoch complete')
    if i==(epochs-1):
      model_x.save("drive/MyDrive/latest"+".h5")



In [ ]:
train_model(8,3)

In [ ]:
from keras.models import load_model
model_x=load_model('/content/drive/MyDrive/imgcap_5.h5')


In [ ]:
def predict_caption(photo):
  ans="<s>"

  for i in range(max_len):
    seq=[word_idx[x] for x in ans.split() if x in word_idx]
    seq=pad_sequences([seq],maxlen=max_len,padding='post')

    y=model_x.predict([photo,seq])
    y_final=y.argmax()
    word=idx_word[y_final]

    ans+=(" "+word)
    if(word=="<e>"):
      break
  
  ans=ans.split()
  caption=ans[1:-1]
  caption=" ".join(caption)
  
  return caption

In [ ]:
                                                         # some manual testing
photo_list=list(train_encoded.keys())
import matplotlib.pyplot as plt


for i in range(6):
  num=np.random.randint(0,1000)
  img_num=photo_list[num]

  img_encoding=train_encoded[img_num].reshape(1,2048)
  cap=predict_caption(img_encoding)

  image=plt.imread("/content/Images/"+img_num+".jpg")
  plt.imshow(image)
  plt.axis("off")
  plt.show()
  print(cap)





  

